In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from resourceReport import ResourceReport

In [64]:
instance = ResourceReport()
df = instance.camp_df

In [65]:
table = instance.master_resource_stats()
print(table)   ### our input pivot camp
camps_with_resources = table.columns[4:] ### range of campIDs
print(camps_with_resources)

    resourceID                      name  priorityLvl  unallocTotal     9  \
0            1                  Blankets            2             0   838   
1            2                  Clothing            1             0   240   
2            3                      Toys            3             0   109   
3            4                  Medicine            1             0    15   
4            5                  Vitamins            2             0    86   
5            6                      Food            1             0    13   
6            7                    Snacks            3             0    10   
7            8                Toiletries            2             0    11   
8            9                     Water            1             0    21   
9           10             Baby Supplies            1             0    13   
10          11  General Tools & Supplies            3             0  4144   

      11     26     49  
0    419   2094   5025  
1    120    599   1438  


In [66]:
col_names = table.columns[4:].tolist()
for col in table.columns[4:]:
    table = table.rename(columns = {col: 'campID ' + str(col)}) ## the col selects the right column

print(table)


    resourceID                      name  priorityLvl  unallocTotal  campID 9  \
0            1                  Blankets            2             0       838   
1            2                  Clothing            1             0       240   
2            3                      Toys            3             0       109   
3            4                  Medicine            1             0        15   
4            5                  Vitamins            2             0        86   
5            6                      Food            1             0        13   
6            7                    Snacks            3             0        10   
7            8                Toiletries            2             0        11   
8            9                     Water            1             0        21   
9           10             Baby Supplies            1             0        13   
10          11  General Tools & Supplies            3             0      4144   

    campID 11  campID 26  c

In [70]:
num_cols = table.columns[3:]
### getting the horizontal sums
h_sum = table[num_cols].sum(axis=1)
table['total per resource'] = h_sum
print(table)

    resourceID                      name  priorityLvl  unallocTotal  campID 9  \
0            1                  Blankets            2             0       838   
1            2                  Clothing            1             0       240   
2            3                      Toys            3             0       109   
3            4                  Medicine            1             0        15   
4            5                  Vitamins            2             0        86   
5            6                      Food            1             0        13   
6            7                    Snacks            3             0        10   
7            8                Toiletries            2             0        11   
8            9                     Water            1             0        21   
9           10             Baby Supplies            1             0        13   
10          11  General Tools & Supplies            3             0      4144   

    campID 11  campID 26  c

In [71]:
v_sum = table[num_cols].sum().tolist()
## need to create a row, thats [NA, NA, 'total per camp', [v,sum]]
row_builder_sum = [np.nan, np.nan, 'total per camp ->'] + v_sum
print(row_builder_sum)
new_row_df = pd.DataFrame([row_builder_sum], columns=table.columns)
# Append the new DataFrame to the original
table = pd.concat([table, new_row_df], ignore_index=True)
print(table)

[nan, nan, 'total per camp ->', 0, 5500, 2749, 13746, 32987, 164946]
    resourceID                      name        priorityLvl  unallocTotal  \
0          1.0                  Blankets                  2             0   
1          2.0                  Clothing                  1             0   
2          3.0                      Toys                  3             0   
3          4.0                  Medicine                  1             0   
4          5.0                  Vitamins                  2             0   
5          6.0                      Food                  1             0   
6          7.0                    Snacks                  3             0   
7          8.0                Toiletries                  2             0   
8          9.0                     Water                  1             0   
9         10.0             Baby Supplies                  1             0   
10        11.0  General Tools & Supplies                  3             0   
11     

In [72]:
df = instance.camp_df

condition = df['campID'].isin(camps_with_resources)
status_vector = df.loc[condition, 'status'].tolist()
status_vector = [np.nan, np.nan, 'camp status ->'] + [np.nan] + status_vector + [np.nan]
print(status_vector)

pop_vector = df.loc[condition, 'refugeePop'].tolist()
pop_vector = [np.nan, np.nan, 'refugeePop ->'] + [np.nan] + pop_vector + [np.nan]
print(pop_vector)

[nan, nan, 'camp status ->', nan, 'open', 'open', 'open', 'open', nan]
[nan, nan, 'refugeePop ->', nan, 2, 1, 5, 12, nan]


In [73]:
top_rows_df = pd.DataFrame([pop_vector, status_vector], columns=table.columns)
# Append the new DataFrame to the original
table = pd.concat([table, top_rows_df], ignore_index=True)
print(table)

    resourceID                      name        priorityLvl  unallocTotal  \
0          1.0                  Blankets                  2           0.0   
1          2.0                  Clothing                  1           0.0   
2          3.0                      Toys                  3           0.0   
3          4.0                  Medicine                  1           0.0   
4          5.0                  Vitamins                  2           0.0   
5          6.0                      Food                  1           0.0   
6          7.0                    Snacks                  3           0.0   
7          8.0                Toiletries                  2           0.0   
8          9.0                     Water                  1           0.0   
9         10.0             Baby Supplies                  1           0.0   
10        11.0  General Tools & Supplies                  3           0.0   
11         NaN                       NaN  total per camp ->           0.0   

In [74]:
print(table.fillna('').to_string(index=False))

resourceID                     name       priorityLvl unallocTotal campID 9 campID 11 campID 26 campID 49 total per resource
       1.0                 Blankets                 2          0.0      838       419      2094      5025            25128.0
       2.0                 Clothing                 1          0.0      240       120       599      1438             7191.0
       3.0                     Toys                 3          0.0      109        54       272       653             3264.0
       4.0                 Medicine                 1          0.0       15         8        38        92              459.0
       5.0                 Vitamins                 2          0.0       86        43       214       513             2568.0
       6.0                     Food                 1          0.0       13         6        32        76              381.0
       7.0                   Snacks                 3          0.0       10         5        24        57              288.0


In [85]:
single_resource = pd.concat([table.tail(2),table[table['resourceID'] == 5]])
print(single_resource.fillna(''))

   resourceID      name     priorityLvl unallocTotal campID 9 campID 11  \
12                        refugeePop ->                     2         1   
13                       camp status ->                  open      open   
4         5.0  Vitamins               2          0.0       86        43   

   campID 26 campID 49 total per resource  
12         5        12                     
13      open      open                     
4        214       513             2568.0  


In [76]:
# Sample DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': [5, 6, 7, 8],
    'C': [9, 10, 11, 12]
})

# Function to print a row in blue
def print_blue_row(row):
    blue_start = "\033[34m"  # ANSI code for blue text
    blue_end = "\033[0m"  # ANSI code to reset color
    row_str = ' '.join(str(item).ljust(8) for item in row)  # Adjust spacing as needed
    print(blue_start + row_str + blue_end)

# Function to print the DataFrame with certain rows in blue
def print_df_with_blue_rows(df, blue_rows):
    # Print column headers
    print(' '.join(col.ljust(8) for col in df.columns))
    
    # Iterate through DataFrame rows
    for index, row in df.iterrows():
        if index in blue_rows:
            print_blue_row(row)
        else:
            print(' '.join(str(item).ljust(8) for item in row))

# Specify the rows you want to print in blue (by index)
blue_rows = [1, 3]  # For example, rows 1 and 3

# Print the DataFrame
print_df_with_blue_rows(df, blue_rows)


A        B        C       
1        5        9       
2        6        10      
3        7        11      
4        8        12      
